Importing libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras import Sequential # type: ignore
import tensorflow as tf

2024-06-06 16:12:44.343150: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 16:12:44.345559: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 16:12:44.373756: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 16:12:45.009835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading the dataset and splitting it to feature and target variables (x, y)

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

LabelEncode the gender column because it has binary values

In [3]:
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

Applying OneHotEncode to the feature variable to get rid of the categorical values and converting them into numerical values

In [4]:
ct = ColumnTransformer([('encoder',OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
x = np.asarray(x).astype(np.float32)

Splitting the feature and the target data into train and test sets

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state = 0)
x_train = np.asarray(x_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

Scaling the features because when we use an ANN model it needs to get scaled to get better results

In [6]:
sc = StandardScaler()
x_train, x_test = sc.fit_transform(x_train), sc.transform(x_test)

Creating 3 hidden layer and one output layer because the prediction we wanna make is classification and not regression

In [7]:
ann = Sequential()
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 416us/step - accuracy: 0.7924 - loss: 0.5536
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - accuracy: 0.7957 - loss: 0.4770
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - accuracy: 0.7959 - loss: 0.4456
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.8012 - loss: 0.4386
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.7968 - loss: 0.4361
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.7990 - loss: 0.4334
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 388us/step - accuracy: 0.8037 - loss: 0.4318
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - accuracy: 0.8201 - loss: 0.4141
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - accuracy: 0.8181 - loss: 0.4251
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - accuracy: 0.8246 - loss: 0.4096
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 381us/step - accuracy: 0.8203 - loss: 0.4216
Epoch 12/100
250/25

Using ann.predict method to predict the test set (which is x_test)

In [8]:
y_ann_pred = ann.predict(x_test)
y_ann_pred_binary = (y_ann_pred >= 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step


Evaluating the model

In [9]:
print(accuracy_score(y_test, y_ann_pred_binary))
print(confusion_matrix(y_test, y_ann_pred_binary))

0.859
[[1507   88]
 [ 194  211]]
